In [1]:
from cftime import DatetimeNoLeap
from datetime import timedelta
from numpy import append, datetime64, empty, float32, full, repeat, stack
from os import chdir, system
from scipy.interpolate import griddata
from warnings import simplefilter
from xarray import DataArray, open_dataset
simplefilter("ignore")

In [18]:
start_year = 2001
start_month = 1
start_day = 1

In [19]:
end_year = 2001
end_month = 1
end_day = 2

**paths**

In [20]:
nc_path = '/home/zhangc/cesm2_cmip6/nc_data/'
im_path = '/home/zhangc/cesm2_cmip6/im_data/'
os_path = '/home/zhangc/repositories/nc2im_cesm2/'

**data**

In [21]:
ncs = ['ta_6hrLev_CESM2_historical_r11i1p1f1_gn_200001010000-200912311800.nc',  #  0
       'hus_6hrLev_CESM2_historical_r11i1p1f1_gn_200001010000-200912311800.nc', #  1
       'ua_6hrLev_CESM2_historical_r11i1p1f1_gn_200001010000-200912311800.nc',  #  2
       'va_6hrLev_CESM2_historical_r11i1p1f1_gn_200001010000-200912311800.nc',  #  3
       'tos_Oday_CESM2_historical_r11i1p1f1_gn_20000102-20150101.nc',           #  4
       'siconc_SIday_CESM2_historical_r11i1p1f1_gn_20000102-20150101.nc',       #  5
       'snw_day_CESM2_historical_r11i1p1f1_gn_20000101-20150101.nc',            #  6
       'ts_Amon_CESM2_historical_r11i1p1f1_gn_200001-201412.nc',                #  7
       'soil_mon_ERA5_2015.nc',                                                 #  8
       'fracdata_0.9x1.25_gx1v6_c090317.nc',                                    #  9
       'USGS-gtopo30_0.9x1.25_remap_c051027.nc']                                # 10

In [22]:
chdir(nc_path)

In [23]:
dss = []
for nc in ncs:
    ds = open_dataset(nc)
    dss.append(ds)

**time**

In [24]:
start_date = DatetimeNoLeap(start_year, start_month, start_day)
end_date = DatetimeNoLeap(end_year, end_month, end_day)

In [25]:
dates = []
date = start_date

In [26]:
while date <= end_date:
    dates.append(date)
    date += timedelta(hours=6)

In [27]:
date = dates[0]

In [28]:
tnum = date.year, date.month, date.day, date.hour
t6hr = DatetimeNoLeap(tnum[0], tnum[1], tnum[2], tnum[3])
tday = DatetimeNoLeap(tnum[0], tnum[1], tnum[2]) + timedelta(days=1)
if tnum[1] is 2:
    mid_day = 14
else:
    mid_day = 15
if tnum[1] in [1, 3, 5, 7, 8, 10, 12]:
    mid_hour = 12
else:
    mid_hour = 0
tmon = DatetimeNoLeap(tnum[0], tnum[1], mid_day, mid_hour)
tera = datetime64('2015-'+str(tnum[1]).zfill(2)+'-01')
file_time = str(t6hr).replace(' ', '_')

**land-sea mask**

In [ ]:
v = dss[9].LANDMASK.values
da = DataArray(name='ls', data=float32(v))
file_name = 'ls_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**surface geopotential**

In [ ]:
v = dss[10].PHIS.values
da = DataArray(name='phis', data=float32(v))
file_name = 'phis_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**soil height**

In [ ]:
v = dss[10].PHIS.values/9.80655
da = DataArray(name='sh', data=float32(v))
file_name = 'sh_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**basic info**

In [240]:
ds = dss[0].sel(time=t6hr)

In [241]:
da = ds.ta
file_name = 'time_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

In [242]:
ss = ['lon', 'lat', 'ps']

In [215]:
for s in ss:  
    v = ds[s].values
    da = DataArray(name=s, data=float32(v))
    file_name = s + '_' + file_time + '.nc'
    da.to_netcdf(im_path+file_name)

In [216]:
v = ds.a.values
da = DataArray(name='hyam', data=float32(v))
file_name = 'hyam' + '_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

In [217]:
v = ds.b.values
da = DataArray(name='hybm', data=float32(v))
file_name = 'hybm' + '_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

In [228]:
v0 = ds.a_bnds.values[:,0]
v1 = ds.a_bnds.values[:,1]
v = append(v1, v0[-1])
da = DataArray(name='hyai', data=float32(v))
da.to_netcdf(im_path+file_name)

In [230]:
v0 = ds.b_bnds.values[:,0]
v1 = ds.b_bnds.values[:,1]
v = append(v1, v0[-1])
da = DataArray(name='hybi', data=float32(v))
da.to_netcdf(im_path+file_name)

**3d temperature**

In [ ]:
v = ds.ta.values
da = DataArray(name='ta', data=float32(v))
file_name = 'ta_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**3d humidity, winds**

In [ ]:
for i in range(1, 4):
    ds = dss[i].sel(time=t6hr)
    vi = ds.variable_id
    v = ds[vi].values
    da = DataArray(name=vi, data=float32(v))
    file_name = vi + '_' + file_time + '.nc'
    da.to_netcdf(im_path+file_name)

**3d virtual temperature**

In [ ]:
t = dss[0].ta.sel(time=t6hr).values
q = dss[1].hus.sel(time=t6hr).values
tv = t*(1.+q*0.61)
da = DataArray(name='tv', data=float32(tv))
file_name = 'tv_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**skin temperature**

In [ ]:
v = dss[7].ts.sel(time=tmon).values
da = DataArray(name='ts', data=float32(v))
file_name = 'ts_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**sea surface temperature**

In [ ]:
ds = dss[4].tos.sel(time=tday)
x = ds.lon.values.flatten()
y = ds.lat.values.flatten()
xy = stack((x, y), axis=-1)
v = ds.values.flatten()

In [ ]:
lon1d = dss[0].lon.values
lat1d = dss[0].lat.values
nx = len(lon1d)
ny = len(lat1d)
lon2d = repeat(lon1d, ny).reshape(nx, ny).transpose()
lat2d = repeat(lat1d, nx).reshape(ny, nx)

In [ ]:
vi = griddata(xy, v, (lon2d, lat2d), method='linear')
da = DataArray(name='sst', data=float32(vi))
file_name = 'sst_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**sea ice concentration**

In [ ]:
da = dss[5].siconc.sel(time=tday)
v = da.values.flatten()
vi = griddata(xy, v, (lon2d, lat2d), method='linear')
da = DataArray(name='sic', data=float32(vi))
file_name = 'sic_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**surface snow amount**

In [ ]:
v = dss[6].snw.sel(time=tday).values
da = DataArray(name='snw', data=float32(v))
file_name = 'snw_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**soil moisture, temperature**

In [ ]:
ds = dss[8].sel(time=tera)
ss = ['swvl1', 'swvl2', 'swvl3', 'swvl4', 'stl1', 'stl2', 'stl3', 'stl4']

In [ ]:
for s in ss:
    v = ds[s].interp(longitude=lon1d, latitude=lat1d).values
    da = DataArray(name=s, data=float32(v))
    file_name = s + '_' + file_time + '.nc'
    da.to_netcdf(im_path+file_name)

In [ ]:
#chdir(os_path)

In [ ]:
#command = 'ncl convert_nc_to_im.ncl ' + "'file_time=" + '"' + file_time + '"' + "'"

In [ ]:
#system(command)